<img src="../images/ARCO.jpeg" width=800 alt="Project Pythia Logo"></img>

# Analysis-Ready Cloud-Optimized Datasets

---

## Overview
In this notebook, we will explore Analysis-Ready Cloud-Optimized (ARCO) radar datasets using Canadian weather radar data. You'll learn:

1. **Analysis-Ready datasets** - Pre-processed data ready for immediate analysis
2. **Cloud-Optimized formats** - Efficient storage and access in cloud environments  
3. **FAIR principles** - Making data Findable, Accessible, Interoperable, and Reusable
4. **Zarr format** - Modern chunked storage for large scientific datasets

We'll use Canadian radar data from the May 2022 Ontario Derecho severe weather event.

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Xarray](https://foundations.projectpythia.org/core/xarray) | Necessary |  Basic features |
| [Radar Cookbook](https://projectpythia.org/radar-cookbook/) | Necessary |  Radar basics |
|[Intro to Zarr](https://zarr.readthedocs.io/en/stable/tutorial.html)| Necessary | Zarr basics

- **Time to learn**: 30 minutes


---

## Imports


In [1]:
import xarray as xr
import fsspec
from glob import glob
import xradar as xd
import matplotlib.pyplot as plt
import cmweather
import numpy as np
import hvplot.xarray
from xarray.core.datatree import DataTree
from zarr.errors import ContainsGroupError  

## Analysis-Ready Data

**Analysis-Ready data** means datasets are prepared and structured to be immediately usable for scientific analysis. Studies show that data scientists typically spend ~80% of their time preparing and cleaning data rather than doing actual analysis.

Analysis-Ready datasets solve this by providing:
- **Clean, pre-processed data** that's ready to use
- **Rich metadata** that explains what the data contains
- **Standardized formats** that work well with analysis tools
- **Quality control** that ensures data reliability

This means more time for science and discovery! 🚀

<img src="../images/AR.jpeg" width=400 alt="Project Pythia Logo"></img>

**Key Benefits of Analysis-Ready Data:**

✅ **Datasets instead of scattered files** - Organized collections of related data  
✅ **Pre-processed and clean** - No need to spend hours fixing data issues  
✅ **Rich metadata included** - Clear documentation of what the data represents  
✅ **Cataloged and discoverable** - Easy to find relevant datasets  
✅ **Immediate analysis capability** - Start analyzing right away  
✅ **More time for science!** - Focus on research questions, not data wrangling

## Cloud-Optimized Data

Traditional radar data formats (like individual NetCDF files) work well on local computers but are slow and inefficient in cloud environments. **Cloud-Optimized** formats like Zarr are designed specifically for fast, efficient access from cloud storage.

![Move to cloud diagram](../images/cloud-move.png)

**Why Cloud-Optimized matters:**
- **Parallel access** - Multiple users can read different parts simultaneously
- **Chunked storage** - Only download the data you need
- **Fast streaming** - No need to download entire files
- **Scalable processing** - Handle datasets too large for local computers

## FAIR Data Principles

**FAIR** data follows principles that make scientific data more valuable and reusable:

- **F**indable - Easy to discover through catalogs and search
- **A**ccessible - Available through standard protocols  
- **I**nteroperable - Works with different tools and systems
- **R**eusable - Well-documented for future use by others

<!-- ![fair](../images/fair-data-principles.jpg) -->
<img src="../images/fair-data-principles.jpg" width=600 alt="Fair"></img>

**FAIR data benefits everyone:**
- **Data producers** get citations when others use their datasets
- **Data consumers** access interesting datasets that would otherwise be unavailable
- **Science advances** through improved data sharing and collaboration

<img src="../images/fair-reuse-300dpi.png" width=400 alt="Fair"></img>

*Image courtesy: [Zarr illustrations](https://github.com/zarr-developers/zarr-illustrations-falk-2022)*

## Zarr format

**Zarr** is a modern storage format designed for large scientific datasets. Instead of storing data in single large files, Zarr breaks data into small "chunks" that can be:

- **Compressed** to save storage space
- **Accessed in parallel** by multiple users
- **Streamed efficiently** from cloud storage
- **Processed on-demand** without downloading everything

Think of it like having a library where you can grab just the books you need, rather than having to check out the entire library!

<img src="../images/monolithic-vs-chunked-1200dpi.png" width=400 alt="zarr"></img>

Courtesy: [Zarr illustrations](https://github.com/zarr-developers/zarr-illustrations-falk-2022?tab=readme-ov-file)

## Creating ARCO Radar Datasets

We'll create Analysis-Ready Cloud-Optimized radar datasets using the **CfRadial2.1/FM301 standard** - a hierarchical structure endorsed by the World Meteorological Organization (WMO). This standard organizes radar data efficiently for both storage and analysis.

### CfRadial2.1/FM301 standard

The **DataTree structure** organizes radar data hierarchically:
- **Root level**: Contains general radar metadata (location, time, etc.)
- **Sweep levels**: Each elevation angle gets its own dataset with radar variables
- **This structure** mirrors how meteorologists think about radar scans

<img src="../images/CfRadial2.1.svg" width=500 alt="xradar"></img>

Let's explore how this works with Canadian radar data from the Ontario Derecho event

In [2]:
# Connect to AMS 2025 data bucket and find Canadian radar data
URL = 'https://js2.jetstream-cloud.org:8001/'
path = f'pythia/radar/ams2025'

fs = fsspec.filesystem("s3", anon=True, client_kwargs=dict(endpoint_url=URL))

# Find Canadian ODIM_H5 radar files from Ontario Derecho 2022
path = "pythia/radar/ams2025/OntarioDerecho2022/*.h5"
radar_files = fs.glob(path)
print(f"Found {len(radar_files)} Canadian radar files")
radar_files[:3]

Found 78 Canadian radar files


['pythia/radar/ams2025/OntarioDerecho2022/2022052115_00_ODIMH5_PVOL6S_VOL_CASET.h5',
 'pythia/radar/ams2025/OntarioDerecho2022/2022052115_00_ODIMH5_PVOL6S_VOL_CASKR.h5',
 'pythia/radar/ams2025/OntarioDerecho2022/2022052115_06_ODIMH5_PVOL6S_VOL_CASET.h5']

In [3]:
# Set up local file caching for efficient access
# We'll use just a few files for this educational example
local_files = [
    fsspec.open_local(
        f"simplecache::{URL}{i}", s3={"anon": True}, filecache={"cache_storage": "."}
    )
    for i in radar_files[:3]  # Use first 3 files to keep example simple
]
print(f"Using {len(local_files)} files for this example")

Using 3 files for this example


Let's open one Canadian radar file (ODIM_H5 format) and see the hierarchical structure

In [4]:
# Open Canadian radar data using xradar's ODIM_H5 reader
dt = xd.io.open_odim_datatree(local_files[0])
print("🇨🇦 Canadian radar data loaded successfully!")
print(f"Radar location: {dt.latitude.values:.2f}°N, {dt.longitude.values:.2f}°W")
print(f"Number of elevation sweeps: {len([k for k in dt.children if 'sweep' in k])}")
display(dt)

🇨🇦 Canadian radar data loaded successfully!
Radar location: 43.37°N, -81.38°W
Number of elevation sweeps: 17


<xarray.DataTree>
Group: /
│   Dimensions:              (sweep: 17)
│   Dimensions without coordinates: sweep
│   Data variables:
│       volume_number        int64 8B 0
│       platform_type        <U5 20B 'fixed'
│       instrument_type      <U5 20B 'radar'
│       time_coverage_start  <U20 80B '2022-05-21T14:54:03Z'
│       time_coverage_end    <U20 80B '2022-05-21T14:59:48Z'
│       longitude            float64 8B -81.38
│       altitude             float64 8B 365.4
│       latitude             float64 8B 43.37
│       sweep_fixed_angle    (sweep) float64 136B 24.4 21.2 18.0 ... 1.2 0.8 0.4
│       sweep_group_name     (sweep) int64 136B 0 1 2 3 4 5 6 ... 11 12 13 14 15 16
│   Attributes:
│       Conventions:      ODIM_H5/V2_2
│       instrument_name:  None
│       version:          None
│       title:            None
│       institution:      None
│       references:       None
│       source:           None
│       history:          None
│       comment:          im/exported using xradar
├── Group: /sweep_0
│       Dimensions:            (azimuth: 360, range: 245)
│       Coordinates:
│         * azimuth            (azimuth) float64 3kB 0.4945 1.502 2.507 ... 358.5 359.5
│           elevation          (azimuth) float64 3kB ...
│           time               (azimuth) datetime64[ns] 3kB 2022-05-21T14:54:06.58332...
│         * range              (range) float32 980B 250.0 750.0 ... 1.218e+05 1.222e+05
│           longitude          float64 8B ...
│           latitude           float64 8B ...
│           altitude           float64 8B ...
│       Data variables: (12/15)
│           DBZH               (azimuth, range) float64 706kB ...
│           TH                 (azimuth, range) float64 706kB ...
│           RHOHV              (azimuth, range) float64 706kB ...
│           UPHIDP             (azimuth, range) float64 706kB ...
│           WRADH              (azimuth, range) float64 706kB ...
│           PHIDP              (azimuth, range) float64 706kB ...
│           ...                 ...
│           VRADH              (azimuth, range) float64 706kB ...
│           sweep_mode         <U20 80B ...
│           sweep_number       int64 8B ...
│           prt_mode           <U7 28B ...
│           follow_mode        <U7 28B ...
│           sweep_fixed_angle  float64 8B ...
├── Group: /sweep_1
│       Dimensions:            (azimuth: 360, range: 245)
│       Coordinates:
│         * azimuth            (azimuth) float64 3kB 0.4915 1.497 2.502 ... 358.5 359.5
│           elevation          (azimuth) float64 3kB ...
│           time               (azimuth) datetime64[ns] 3kB 2022-05-21T14:54:21.41666...
│         * range              (range) float32 980B 250.0 750.0 ... 1.218e+05 1.222e+05
│           longitude          float64 8B ...
│           latitude           float64 8B ...
│           altitude           float64 8B ...
│       Data variables: (12/15)
│           DBZH               (azimuth, range) float64 706kB ...
│           TH                 (azimuth, range) float64 706kB ...
│           RHOHV              (azimuth, range) float64 706kB ...
│           UPHIDP             (azimuth, range) float64 706kB ...
│           WRADH              (azimuth, range) float64 706kB ...
│           PHIDP              (azimuth, range) float64 706kB ...
│           ...                 ...
│           VRADH              (azimuth, range) float64 706kB ...
│           sweep_mode         <U20 80B ...
│           sweep_number       int64 8B ...
│           prt_mode           <U7 28B ...
│           follow_mode        <U7 28B ...
│           sweep_fixed_angle  float64 8B ...
├── Group: /sweep_2
│       Dimensions:            (azimuth: 360, range: 245)
│       Coordinates:
│         * azimuth            (azimuth) float64 3kB 0.5055 1.511 2.502 ... 358.5 359.5
│           elevation          (azimuth) float64 3kB ...
│           time               (azimuth) datetime64[ns] 3kB 2022-05-21T14:54:36.21665...
│         * range              (range) f

Now let's create our first **ARCO dataset** by converting this radar data to Zarr format

In [5]:
# Convert to ARCO format using Zarr
dt.to_zarr("canadian_radar_arco.zarr", consolidated=True)
print("✅ Canadian radar data converted to ARCO format!")
print("📁 This creates a cloud-optimized, analysis-ready dataset")

✅ Canadian radar data converted to ARCO format!
📁 This creates a cloud-optimized, analysis-ready dataset


Let's verify our ARCO dataset was created successfully

In [6]:
# Check that our ARCO dataset was created
!ls -la *.zarr

total 520
drwxrwxrwx 1 alfonso alfonso    488 Aug 20 13:49 .
drwxrwxrwx 1 alfonso alfonso    456 Aug 20 13:49 ..
drwxrwxrwx 1 alfonso alfonso      0 Aug 20 13:49 altitude
drwxrwxrwx 1 alfonso alfonso      0 Aug 20 13:49 georeferencing_correction
drwxrwxrwx 1 alfonso alfonso      0 Aug 20 13:49 instrument_type
drwxrwxrwx 1 alfonso alfonso      0 Aug 20 13:49 latitude
drwxrwxrwx 1 alfonso alfonso      0 Aug 20 13:49 longitude
drwxrwxrwx 1 alfonso alfonso      0 Aug 20 13:49 platform_type
drwxrwxrwx 1 alfonso alfonso      0 Aug 20 13:49 radar_calibration
drwxrwxrwx 1 alfonso alfonso      0 Aug 20 13:49 radar_parameters
drwxrwxrwx 1 alfonso alfonso   4096 Aug 20 13:49 sweep_0
drwxrwxrwx 1 alfonso alfonso   4096 Aug 20 13:49 sweep_1
drwxrwxrwx 1 alfonso alfonso   4096 Aug 20 13:49 sweep_10
drwxrwxrwx 1 alfonso alfonso   4096 Aug 20 13:49 sweep_11
drwxrwxrwx 1 alfonso alfonso   4096 Aug 20 13:49 sweep_12
drwxrwxrwx 1 alfonso alfonso   4096 Aug 20 13:49 sweep_13
drwxrwxrwx 1 alfonso alfonso  

The Zarr format creates a directory structure that can be stored locally or in cloud storage. Let's read our ARCO dataset back to confirm it works

In [7]:
# Read our ARCO dataset back from Zarr storage
dt_back = xr.backends.api.open_datatree(
    "canadian_radar_arco.zarr", 
    consolidated=True
)
print("🎉 Successfully opened ARCO dataset!")
print("⚡ This would work the same way if stored in cloud storage")

🎉 Successfully opened ARCO dataset!
⚡ This would work the same way if stored in cloud storage


In [8]:
display(dt_back)
print("\n✅ Perfect! The data structure is preserved in ARCO format")

<xarray.DataTree>
Group: /
│   Dimensions:              (sweep: 17)
│   Dimensions without coordinates: sweep
│   Data variables:
│       latitude             float64 8B ...
│       altitude             float64 8B ...
│       longitude            float64 8B ...
│       platform_type        <U5 20B ...
│       instrument_type      <U5 20B ...
│       sweep_fixed_angle    (sweep) float64 136B ...
│       sweep_group_name     (sweep) int64 136B ...
│       volume_number        int64 8B ...
│       time_coverage_start  <U20 80B ...
│       time_coverage_end    <U20 80B ...
│   Attributes:
│       Conventions:      ODIM_H5/V2_2
│       instrument_name:  None
│       version:          None
│       title:            None
│       institution:      None
│       references:       None
│       source:           None
│       history:          None
│       comment:          im/exported using xradar
├── Group: /georeferencing_correction
│       Dimensions:    ()
│       Coordinates:
│           altitude   float64 8B ...
│           latitude   float64 8B ...
│           longitude  float64 8B ...
├── Group: /radar_parameters
│       Dimensions:    ()
│       Coordinates:
│           altitude   float64 8B ...
│           latitude   float64 8B ...
│           longitude  float64 8B ...
├── Group: /sweep_0
│       Dimensions:            (azimuth: 360, range: 245)
│       Coordinates:
│           altitude           float64 8B ...
│         * azimuth            (azimuth) float64 3kB 0.4945 1.502 2.507 ... 358.5 359.5
│           elevation          (azimuth) float64 3kB ...
│           latitude           float64 8B ...
│           longitude          float64 8B ...
│         * range              (range) float32 980B 250.0 750.0 ... 1.218e+05 1.222e+05
│           time               (azimuth) datetime64[ns] 3kB ...
│       Data variables: (12/15)
│           DBZH               (azimuth, range) float64 706kB ...
│           follow_mode        <U7 28B ...
│           prt_mode           <U7 28B ...
│           PHIDP              (azimuth, range) float64 706kB ...
│           KDP                (azimuth, range) float64 706kB ...
│           RHOHV              (azimuth, range) float64 706kB ...
│           ...                 ...
│           UPHIDP             (azimuth, range) float64 706kB ...
│           sweep_mode         <U20 80B ...
│           sweep_number       int64 8B ...
│           TH                 (azimuth, range) float64 706kB ...
│           WRADH              (azimuth, range) float64 706kB ...
│           ZDR                (azimuth, range) float64 706kB ...
...
├── Group: /sweep_8
│       Dimensions:            (azimuth: 360, range: 480)
│       Coordinates:
│           elevation          (azimuth) float64 3kB ...
│         * azimuth            (azimuth) float64 3kB 0.4915 1.516 2.505 ... 358.5 359.5
│           latitude           float64 8B ...
│           altitude           float64 8B ...
│           longitude          float64 8B ...
│         * range              (range) float32 2kB 250.0 750.0 ... 2.392e+05 2.398e+05
│           time               (azimuth) datetime64[ns] 3kB ...
│       Data variables: (12/15)
│           DBZH               (azimuth, range) float64 1MB ...
│           KDP                (azimuth, range) float64 1MB ...
│           follow_mode        <U7 28B ...
│           prt_mode           <U7 28B ...
│           PHIDP              (azimuth, range) float64 1MB ...
│           sweep_fixed_angle  float64 8B ...
│           ...                 ...
│           sweep_number       int64 8B ...
│           sweep_mode         <U20 80B ...
│           VRADH              (azimuth, range) float64 1MB ...
│           WRADH              (azimuth, range) float64 1MB ...
│           ZDR                (azimuth, range) float64 1MB ...
│           UPHIDP             (azimuth, range) float64 1MB ...
├── Group: /sweep_9
│       Dimensions:            (azimuth: 360, range: 480)
│       Coordinates:
│           altitude           float


✅ Perfect! The data structure is preserved in ARCO format


## Creating Radar Time Series

For weather analysis, we often want to track storms over time. Let's create a **time series dataset** by combining multiple radar volumes from the Ontario Derecho event. This shows how storms evolved during this severe weather outbreak.

In [9]:
# Check how many files we're working with
print(f"📊 Working with {len(local_files)} radar volumes for time series")
print("This represents radar scans taken minutes apart during the derecho")

📊 Working with 3 radar volumes for time series
This represents radar scans taken minutes apart during the derecho


To create a proper time series, all radar volumes need to be **aligned** - meaning they have the same azimuth angles and range gates. This is essential for tracking storm features over time.

In [10]:
def fix_angle(ds: xr.Dataset, tolerance: float=None, **kwargs) -> xr.Dataset:
    """
    Align radar azimuth angles to create consistent time series data.
    
    This function ensures all radar sweeps have the same azimuth grid,
    which is essential for tracking storm evolution over time.
    
    Parameters:
    -----------
    ds : xarray Dataset
        Radar sweep dataset
    tolerance : float, optional
        Angular tolerance for alignment (uses radar resolution if not specified)
    
    Returns:
    --------
    xarray Dataset
        Aligned radar dataset ready for time series analysis
    """
    # Load time data efficiently
    ds["time"] = ds.time.load() 
    
    # Extract radar scanning parameters
    angle_dict = xd.util.extract_angle_parameters(ds)
    start_ang = angle_dict["start_angle"]
    stop_ang = angle_dict["stop_angle"] 
    direction = angle_dict["direction"]
    
    # Remove any duplicate rays (quality control)
    ds = xd.util.remove_duplicate_rays(ds)
    
    # Calculate azimuth resolution
    az_range = len(np.arange(start_ang, stop_ang))
    angular_resolution = np.round(az_range / len(ds.azimuth.data), 2)
    tolerance = angular_resolution if not tolerance else tolerance
    
    # Align to common azimuth grid
    ds = xd.util.reindex_angle(
        ds, start_ang, stop_ang, angular_resolution, direction, 
        method="nearest", tolerance=tolerance, **kwargs
    )
    
    return ds

Now we'll create the time series dataset by processing each elevation sweep separately and combining the time dimension

In [12]:
# Create time series ARCO dataset
# For educational purposes, we'll process just a few sweeps to keep it manageable
print("🔄 Creating time series ARCO dataset...")
print("This combines multiple radar volumes along the time dimension")
dict_tree = {}
# Process first 3 elevation sweeps to keep example manageable
for i, sweep in enumerate(sweeps[:3]):
    print(f"📊 Processing {sweep}...")
    
    # Use xarray's powerful multi-file capabilities
    ds = xr.open_mfdataset(
        local_files,
        preprocess=fix_angle,        # Align all volumes  
        engine="odim",               # Use ODIM_H5 engine
        group=sweep,                 # Process this elevation
        concat_dim="volume_time",    # Combine along time
        combine="nested"             # Nested combination
    ).xradar.georeference()          # Add x,y coordinates
    
    # Create DataTree structure
    dict_tree[f"{sweep}"] = ds
    
dtree = DataTree.from_dict(dict_tree)
# Save to ARCO time series format
try:
    dtree.to_zarr("canadian_timeseries.zarr", consolidated=True)
    print(f"  ✅ Added {sweep} to time series")
except ContainsGroupError:
    # Append to existing dataset
    dtree.to_zarr("canadian_timeseries.zarr", consolidated=True, mode="w")
    print(f"  ✅ Appended {sweep} to time series")

# Clean up memory
del dtree, ds
    
print("\n🎉 Time series ARCO dataset created successfully!")
print("📈 This dataset now contains storm evolution over time")

🔄 Creating time series ARCO dataset...
This combines multiple radar volumes along the time dimension
📊 Processing sweep_0...
📊 Processing sweep_1...
📊 Processing sweep_2...
  ✅ Added sweep_2 to time series

🎉 Time series ARCO dataset created successfully!
📈 This dataset now contains storm evolution over time


Let's examine our new time series ARCO dataset

In [13]:
# Open our time series ARCO dataset
dtree_ts = xr.backends.api.open_datatree(
    "canadian_timeseries.zarr",
    consolidated=True,
    chunks={}  # Let xarray handle chunking automatically
)
print("⏰ Time series ARCO dataset loaded!")
print("Now we can analyze storm evolution over time")

⏰ Time series ARCO dataset loaded!
Now we can analyze storm evolution over time


In [14]:
display(dtree_ts)
print("\n📊 Notice the 'volume_time' dimension - this allows us to track storms over time!")
print("🔍 Each sweep now contains multiple time steps from the derecho event")

<xarray.DataTree>
Group: /
├── Group: /sweep_1
│       Dimensions:            (volume_time: 3, azimuth: 360, range: 245)
│       Coordinates:
│           altitude           float64 8B ...
│         * azimuth            (azimuth) float64 3kB 0.5 1.5 2.5 ... 357.5 358.5 359.5
│           crs_wkt            int64 8B ...
│           elevation          (azimuth) float64 3kB dask.array<chunksize=(360,), meta=np.ndarray>
│           latitude           float64 8B ...
│           longitude          float64 8B ...
│           time               (azimuth) datetime64[ns] 3kB dask.array<chunksize=(360,), meta=np.ndarray>
│           x                  (azimuth, range) float64 706kB dask.array<chunksize=(180, 123), meta=np.ndarray>
│           z                  (azimuth, range) float64 706kB dask.array<chunksize=(180, 123), meta=np.ndarray>
│         * range              (range) float32 980B 250.0 750.0 ... 1.218e+05 1.222e+05
│           y                  (azimuth, range) float64 706kB dask.array<chunksize=(180, 123), meta=np.ndarray>
│       Dimensions without coordinates: volume_time
│       Data variables: (12/15)
│           KDP                (volume_time, azimuth, range) float64 2MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│           follow_mode        (volume_time) <U7 84B dask.array<chunksize=(3,), meta=np.ndarray>
│           DBZH               (volume_time, azimuth, range) float64 2MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│           PHIDP              (volume_time, azimuth, range) float64 2MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│           SQIH               (volume_time, azimuth, range) float64 2MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│           sweep_fixed_angle  (volume_time) float64 24B dask.array<chunksize=(3,), meta=np.ndarray>
│           ...                 ...
│           TH                 (volume_time, azimuth, range) float64 2MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│           sweep_number       (volume_time) int64 24B dask.array<chunksize=(3,), meta=np.ndarray>
│           WRADH              (volume_time, azimuth, range) float64 2MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│           VRADH              (volume_time, azimuth, range) float64 2MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│           UPHIDP             (volume_time, azimuth, range) float64 2MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│           ZDR                (volume_time, azimuth, range) float64 2MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│       Attributes:
│           Conventions:  ODIM_H5/V2_2
├── Group: /sweep_0
│       Dimensions:            (volume_time: 3, azimuth: 360, range: 245)
│       Coordinates:
│           altitude           float64 8B ...
│         * azimuth            (azimuth) float64 3kB 0.5 1.5 2.5 ... 357.5 358.5 359.5
│           crs_wkt            int64 8B ...
│           elevation          (azimuth) float64 3kB dask.array<chunksize=(360,), meta=np.ndarray>
│           latitude           float64 8B ...
│           longitude          float64 8B ...
│         * range              (range) float32 980B 250.0 750.0 ... 1.218e+05 1.222e+05
│           time               (azimuth) datetime64[ns] 3kB dask.array<chunksize=(360,), meta=np.ndarray>
│           x                  (azimuth, range) float64 706kB dask.array<chunksize=(180, 123), meta=np.ndarray>
│           y                  (azimuth, range) float64 706kB dask.array<chunksize=(180, 123), meta=np.ndarray>
│           z                  (azimuth, range) float64 706kB dask.array<chunksize=(180, 123), meta=np.ndarray>
│       Dimensions without coordinates: volume_time
│       Data variables: (12/15)
│           follow_mode        (volume_time) <U7 84B dask.array<chunksize=(3,), meta=np.ndarray>
│           KDP                (volume_time, azimuth, range) float64 2MB dask.array<chunksize=(1, 360, 245), meta=np.ndarray>
│           PHIDP             


📊 Notice the 'volume_time' dimension - this allows us to track storms over time!
🔍 Each sweep now contains multiple time steps from the derecho event


## ✅ Success! We've created Analysis-Ready Cloud-Optimized datasets

Our ARCO datasets are now ready for immediate scientific analysis with several key advantages:

---

## Summary

We successfully created **Analysis-Ready Cloud-Optimized (ARCO)** radar datasets using Canadian weather data! Here's what we accomplished:

### 🎯 **Key Learning Outcomes:**

**📊 Analysis-Ready**: Pre-processed, clean datasets ready for immediate scientific analysis  
**☁️ Cloud-Optimized**: Efficient Zarr format enabling fast access from cloud storage  
**🌐 FAIR Principles**: Making data Findable, Accessible, Interoperable, and Reusable  
**📈 Time Series**: Combined multiple radar volumes to track storm evolution  
**🏗️ Standardized Structure**: Used WMO-endorsed FM301 hierarchical organization

### 🚀 **What This Enables:**
- **Faster Research**: No more data preprocessing - start analyzing immediately
- **Cloud Analytics**: Process large datasets without downloading everything  
- **Reproducible Science**: Standardized formats work across different tools
- **Collaboration**: Easy data sharing following FAIR principles
- **Storm Tracking**: Time series analysis of severe weather events

### 🔮 **Next Steps:**
Ready to dive deeper? Check out the **QPE-QVPs notebook** to explore:
- Quantitative Precipitation Estimation (QPE) from radar
- Quasi-Vertical Profiles (QVPs) for storm analysis  
- Advanced weather analysis techniques using ARCO datasets

The Ontario Derecho case study demonstrates how ARCO datasets streamline radar meteorology research and education! 🌪️

## Resources and references
 - [Xradar](https://docs.openradarscience.org/projects/xradar/en/stable/index.html)
 - [Radar cookbook](https://github.com/ProjectPythia/radar-cookbook)
 - [Py-Art landing page](https://arm-doe.github.io/pyart/)
 - [Wradlib landing page](https://docs.wradlib.org/en/latest/index.html)